In [2]:
import csv

# Path to raw text file
input_file = "dialogs.txt"
output_file = "dialogue_dataset.csv"

# Open raw file and clean/process
dialogue_pairs = []

with open(input_file, "r", encoding="utf-8") as f:
    for line in f:
        # Skip empty lines
        if not line.strip():
            continue

        # Split by tab
        parts = line.strip().split("\t")
        if len(parts) != 2:
            continue  # Ignore broken lines

        prompt = parts[0].strip()
        response = parts[1].strip()

        # Skip too-short or empty messages
        if len(prompt) < 2 or len(response) < 2:
            continue

        dialogue_pairs.append((prompt, response))

# Write to CSV
with open(output_file, "w", newline='', encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["prompt", "response"])  # header
    writer.writerows(dialogue_pairs)

print(f"✅ Saved {len(dialogue_pairs)} pairs to {output_file}")


✅ Saved 3725 pairs to dialogue_dataset.csv


In [5]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import torch
import random

# Load dataset
df = pd.read_csv("dialogue_dataset.csv")

# Drop nulls or weird entries
df.dropna(inplace=True)
df = df[df["prompt"].str.strip() != ""]
df = df[df["response"].str.strip() != ""]

# Reset index
df.reset_index(drop=True, inplace=True)

# Load pre-trained model (small but powerful)
model = SentenceTransformer('all-MiniLM-L6-v2')

# Encode all prompts
print("Encoding prompts with BERT...")
prompt_embeddings = model.encode(df['prompt'].tolist(), convert_to_tensor=True)

# Chat function
def chat_with_bot(user_input, top_k=1):
    # Encode user input
    user_embedding = model.encode(user_input, convert_to_tensor=True)

    # Compute cosine similarities
    similarities = util.cos_sim(user_embedding, prompt_embeddings)[0]

    # Get top match
    top_idx = torch.argmax(similarities).item()
    matched_prompt = df.iloc[top_idx]['prompt']
    response = df.iloc[top_idx]['response']
    confidence = similarities[top_idx].item()

    print(f"\n🔍 Matched: {matched_prompt} (score={confidence:.4f})")
    return response

# Test it
if __name__ == "__main__":
    print("\n🧠 BERT Chatbot Ready! Type 'exit' to quit.")
    while True:
        user_input = input("\nYou: ")
        if user_input.lower() in ["exit", "quit"]:
            print("Goodbye 👋")
            break
        bot_reply = chat_with_bot(user_input)
        print("Bot:", bot_reply)


Encoding prompts with BERT...


C:\Users\ASUS\AppData\Roaming\Python\Python312\site-packages\torch\nn\modules\module.py:1541: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



🧠 BERT Chatbot Ready! Type 'exit' to quit.

🔍 Matched: hi, how are you doing? (score=0.5620)
Bot: i'm fine. how about yourself?

🔍 Matched: who? (score=0.5372)
Bot: two students from usc.

🔍 Matched: i'm fine. how about yourself? (score=0.5573)
Bot: i'm pretty good. thanks for asking.

🔍 Matched: that's nice. (score=0.6213)
Bot: are you going to be there?

🔍 Matched: no. (score=0.8128)
Bot: how do you know?

🔍 Matched: no. (score=0.8128)
Bot: how do you know?

🔍 Matched: what's wrong? (score=0.6304)
Bot: it's not orange!
Goodbye 👋
